In [34]:
# CLUSTER DEPLOYMENT, TO BE SUBSTITUED WHEN GOING CLUSTER
from dask.distributed import Client, LocalCluster

# For now, local deployment on my computer (multicore)
ncore = 4
cluster = LocalCluster(n_workers=ncore, threads_per_worker=1)
client = Client(cluster)

# Print the dashboard link over the port 8787
print(client.dashboard_link)

/home/lrizzi17/anaconda3/envs/envLabComp/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38813 instead
  warnings.warn(


http://127.0.0.1:38813/status


In [ ]:
# CLUSTER DEPLOYMENT ON CLOUDVENETO
from dask.distributed import Client, SSHCluster

cluster = SSHCluster(
    ["10.67.22.154", "10.67.22.216", "10.67.22.116", "10.67.22.113"],
    connect_options={"known_hosts": None},
    scheduler_options={"port": 8786, "dashboard_address": ":8797"}
)

client = Client(cluster)

In [53]:
# check if everything went smoothly
print(client)

<Client: 'tcp://127.0.0.1:34793' processes=4 threads=4, memory=13.49 GiB>


AttributeError: 'LocalCluster' object has no attribute 'alt'

Import the necessary stuff along with the California housing dataset

In [36]:
import dask.array as da
import dask
import numpy as np
from sklearn.datasets import fetch_california_housing

# Download California Housing dataset
data = fetch_california_housing(as_frame=True)

# Convert features into Dask Array (it's a matrix). N partition = 16
n_partition = 4        # number of partition in memory
length_partition = data.data.shape[0] // n_partition
X_da = da.from_array(data.data.values, chunks=(length_partition, data.data.shape[1]))

print("Number of Dask partitions:",  X_da.npartitions) 
print("Length of each partition:", length_partition, "rows")
print("Length of the whole dataset:", data.data.shape[0], "rows")

Number of Dask partitions: 4
Length of each partition: 5160 rows
Length of the whole dataset: 20640 rows


Now we'll define the parallel and serial algorithm for the Cholesky QR decomposition:

In [37]:
def compute_choleskyQR_parallel(X_da : dask.array.Array):
    # A list of delayed tasks for each partition of the dataset
    # Each partition computes the local Gram matrix
    chunks_delayed = [dask.delayed(lambda x : x.T @ x)(chunk) for chunk in X_da.to_delayed().ravel()]

    # Now sum all the local Gram matrices to get the global Gram matrix
    Gram_global_delayed = dask.delayed(sum)(chunks_delayed)   ## !! This is not strictly parallel, meaning that a single worker will perform the sum instead of a tree-like operation. This is ok here, I guess, since we only have 8 chunks that need to be summed up

    # Compute R as the Cholesky decomposition on the global Gram matrix (as a delayed even if a serial operation just call .compute at the end)
    R = dask.delayed(np.linalg.cholesky)(Gram_global_delayed)
    #R.visualize("fig/CholeskyR.png")
    R = R.compute() # Compute R. This will put a stop at the parallel operation
    R_inv = np.linalg.inv(R) # It's a small matrix, so this operation is fast even if serial

    Q = X_da.map_blocks(lambda block: block @ R_inv, dtype=X_da.dtype)
    #Q.visualize("fig/CholeskyQ.png")
    Q = Q.compute() # Compute Q
    return Q, R

def compute_choleskyQR_serial(X):
    # Global gram matrix
    G = X.T @ X
    R = np.linalg.cholesky(G)
    R_inv = np.linalg.inv(R)
    Q = X @ R_inv
    
    return Q, R

def compute_choleskyR_parallel(X_da : dask.array.Array):
    # A list of delayed tasks for each partition of the dataset
    # Each partition computes the local Gram matrix
    chunks_delayed = [dask.delayed(lambda x : x.T @ x)(chunk) for chunk in X_da.to_delayed().ravel()]
    # Now sum all the local Gram matrices to get the global Gram matrix
    Gram_global_delayed = dask.delayed(sum)(chunks_delayed)
    # Compute R as the Cholesky decomposition on the global Gram matrix (as a delayed even if a serial operation just call .compute at the end)
    R = dask.delayed(np.linalg.cholesky)(Gram_global_delayed)
    R = R.compute() # Compute R
    return  R

def compute_choleskyR_serial(X):
    # Global gram matrix
    G = X.T @ X
    R = np.linalg.cholesky(G)
    return R

Let's measure the time it takes to perform the Cholesky QR decomposition:

In [47]:
%%time
# parallel
Q_p, R_p = compute_choleskyQR_parallel(X_da)

CPU times: user 90.9 ms, sys: 28.9 ms, total: 120 ms
Wall time: 105 ms


The measured wall time is smaller than the total (cpu+system) time, meaning that the program went parallel (and in fact, the ratio wall / total should converge to n_workers)

In [52]:
%%time
# serial
Q_s, R_s = compute_choleskyQR_serial(data.data.values)

CPU times: user 3.38 ms, sys: 435 µs, total: 3.81 ms
Wall time: 2.38 ms


Here again, wall time is smaller than total time. This is because even when using serial Numpy function, under the hood the BLAS implementation of certain algorithm leverages multithreading

However, the serial algorithm is faster than the parallel one. This is probably because the California dataset is not that big (only $20k$ rows, easily fittable in the RAM). As of now, we still can't see a real speedup

Cholesky QR is sadly known to be unstable. In fact:

In [6]:
# Let's see whether the results are compatible
diffR = np.linalg.norm(R_p - R_s, 2)
diffQ = np.linalg.norm(Q_p - Q_s, 2)
print(f"||R_parallel - R_serial||_2 = {diffR}")
print(f"||Q_parallel - Q_serial||_2 = {diffQ}")

# Check orthogonality of Q
orthogonality_metric = np.linalg.norm(Q_s.T @ Q_s - np.eye(Q_s.shape[1]), 2)
print(f"||Q^T @ Q- I||_2 = {orthogonality_metric}")
# Check decomposition
decomp_metric = np.linalg.norm(data.data.values - Q_s @ R_s, 2)
print(f"||X - Q @ R||_2 = {decomp_metric}")

||R_parallel - R_serial||_2 = 6.6703992450451735e-09
||Q_parallel - Q_serial||_2 = 5.281864770776042e-10
||Q^T @ Q- I||_2 = 7971678.680290628
||X - Q @ R||_2 = 2.838161229486053e-10


As expected, the decomposition yielded a non reasonnable result (Q is not orthogonal, the algorithm is highly unstable)


Let's try with a different and larger dataset (HIGGS dataset)

In [7]:
import dask.dataframe as dd

# A huge dataset
df = dd.read_csv("HIGGS.csv", header=None, blocksize="400MB")
X_df = df.iloc[:, 1:] 
X_da = X_df.to_dask_array(lengths=True)

2025-08-27 11:08:01,683 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.07 GiB -- Worker memory limit: 1.69 GiB
2025-08-27 11:08:01,801 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 1.07 GiB -- Worker memory limit: 1.69 GiB
2025-08-27 11:08:01,824 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [ ]:
%%time
Q, R = compute_choleskyQR_parallel(X_da)